In [1]:
from os import path
import argparse
from datetime import datetime
import string
import re
import warnings
warnings.filterwarnings('ignore')
import urllib.request
from itertools import compress

import lxml.html as LH
import requests
from bs4 import BeautifulSoup as bs

import pandas as pd

from utils import get_request

In [6]:
def get_player_info(url):
        r = get_request(url)
        all_tags = bs(r.content, "html.parser")

        tbl = all_tags.find("table", attrs={"class" : "sortable stats_table"})
        tbl_rows = tbl.find_all('tr')
        df_columns = ["player", "pos", "age", "tm", "g", "gs", "mp"]
        line = []
        for tr in tbl_rows:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            line.append(row)
        df = pd.DataFrame(line).iloc[1:,0:7]
        df.columns = df_columns
        df = df.set_index("player")

        ids_raw = [x for x in all_tags.find_all("td", class_ = "left")]
        ids = []
        player_name = []
        for x in ids_raw:
            try:
                player_name.append(x.get_text().strip())
                ids.append(x["data-append-csv"])
            except:
                next
        player_id_df = pd.DataFrame({"player":player_name[::2], "bbrefID":ids}).set_index("player")

        df_combined = df.join(player_id_df, how="inner").reset_index().drop_duplicates()
        return df_combined

In [7]:
test_url = 'https://www.basketball-reference.com/leagues/NBA_2018_per_game.html'
tmp = get_player_info(test_url)

In [25]:
tmp['mp'] = tmp['mp'].apply(lambda x: (int(x.split(".")[0])*60) + int((x.split(".")[1])))

In [26]:
tmp

,player,pos,age,tm,g,gs,mp,bbrefID
0,Aaron Brooks,PG,33,MIN,32,1,309,brookaa01
1,Aaron Gordon,PF,22,ORL,58,57,1929,gordoaa01
2,Aaron Harrison,SG,23,DAL,9,3,1509,harriaa01
3,Aaron Jackson,PG,31,HOU,1,0,2100,jacksaa01
4,Abdel Nader,SF,24,BOS,48,1,609,naderab01
5,Adreian Payne,PF,26,ORL,5,0,486,paynead01
6,Al Horford,C,31,BOS,72,72,1866,horfoal01
7,Al Jefferson,C,33,IND,36,1,784,jeffeal01
8,Al-Farouq Aminu,PF,27,POR,69,67,1800,aminual01
9,Alan Williams,PF,25,PHO,5,0,840,willial03
